In [2]:
import neptune as neptune
from datetime import datetime, timedelta
import pandas as pd
import pytz


In [3]:
def get_training_progress_with_eta(runs_table, run_specs, active=False):
    # Filter runs based on the provided specifications
    for column, value in run_specs.items():
        if value is not None:
            runs_table = runs_table[runs_table[column] == value]
    print(runs_table.shape)

    if runs_table.empty:
        print("No runs found with the specified criteria.")
        return None
    get_training_progress_with_eta
    results = []
    
    for idx, run in runs_table.iterrows():        
        # Fetch necessary information
        exp_id = run['sys/id']
        current_step = run["step"]
        n_steps = run["args/n_steps"]
        start_time = run["sys/creation_time"]
        warsaw_tz = pytz.timezone('Europe/Warsaw')
        current_time = datetime.now(warsaw_tz)
        time_running = current_time - start_time
        time_running = round(time_running.total_seconds() / 3600, 2)
        
        # Calculate the percentage of training completed
        if n_steps > 0:
            percent_finished = (current_step / n_steps) * 100
        else:
            percent_finished = 0.0
        
        # Estimate ETA (remaining time)
        if current_step > 0:
            time_per_step = time_running / current_step
            remaining_steps = n_steps - current_step
            time_left = time_per_step * remaining_steps
            # time_left_hours = time_left.total_seconds() / 3600  # Convert to hours
        else:
            time_left = None
        
        # Collect the data into a dictionary
        result = {
            # "args/name": run.get("args/name", None),
            "ID": exp_id,
            "time_running": time_running,
            "time_left (hours)": time_left,
            "%finished": percent_finished,
            "step": current_step,
            "host": run['sys/hostname']
        }
        
        # Include the run specifications in the result
        for column in run_specs.keys():
            result[column] = run.get(column, None)
        
        results.append(result)
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [4]:
project = neptune.init_project(
    project="pmtest/llm-random",
    mode="read-only",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyMDY0ZDI5Ni05YWU3LTQyNGYtYmY4My1hZTFkY2EzYmUwMjgifQ=="
)

# Find the specific run based on provided specs and tags
# runs_table = project.fetch_runs_table(state="active", tag=["constrained_scaling_grid_final"]).to_pandas()
runs_table = project.fetch_runs_table(tag=["constrained_scaling_grid_final"], state='active').to_pandas()
print(runs_table.shape)

https://app.neptune.ai/pmtest/llm-random/
(0, 0)


In [5]:
model_sizes = {
    'none': None,
    '4M': 3145728,
    '9M': 7077888,
    '33M': 25165824,
    '65M': 49152000,
    '113M': 84934656,
    '260M': 201326592,
    '520M': 393216000,
}

training_lengths = {
    'none': None,
    '500M': 1907,
    '1B': 3814,
    '2B': 7628,
    '4B': 15256,
    '8B': 30512,
    '16B': 61024,
    '32B': 122048,
    '64B': 244150,
    '128B': 488300,
}

In [10]:
def get_neptune_table(tags, negative_tags=None):
    project = neptune.init_project(
        project="pmtest/llm-random",
        mode="read-only",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyMDY0ZDI5Ni05YWU3LTQyNGYtYmY4My1hZTFkY2EzYmUwMjgifQ=="
    )
    runs_table = project.fetch_runs_table(tag=tags).to_pandas()

    # If negative tags are provided, filter them out
    if negative_tags is not None:
        for neg_tag in negative_tags:
            runs_table = runs_table[~runs_table['sys/tags'].apply(lambda x: neg_tag in x)]

    print(f'table downloaded\nshape: {runs_table.shape}')
    return runs_table


def exp_status_by_tags(tags, negative_tags=None):

    runs_table = get_neptune_table(tags=tags, negative_tags=negative_tags)
    print(runs_table.shape)

    run_specs = {"args/n_steps": training_lengths['none'], "args/model_n_active": model_sizes['none'], "args/expansion_rate": None, "sys/state": None}
    active = False

    result = get_training_progress_with_eta(runs_table, run_specs, active=active)
    result["args/model_n_active"] = (result["args/model_n_active"] / 1000000).round().astype(int)
    result["time_left (d)"] = (result["time_left (hours)"] / 24).round(2)
    # result = result[result["%finished"] != 100]

    return result


In [11]:
component_names = ['embedding_layer', 'head', 'gating', 'expert_inner_function', 'projection']
start_ends = ['start', 'end']
negative_tags = ['remove']

uplot_medium_tags = ['relativity_paper', 'large_model', 'std']


# for start_end in start_ends:
#     for component in component_names:
#         uplot_medium_tags = ['relativity_paper', 'medium_model', 'uplot', start_end, component]

#         df = get_neptune_table(tags=uplot_medium_tags, negative_tags=negative_tags)
#         print(f'{start_end}, component: {component}, shape: {df.shape}')

not_finished = exp_status_by_tags(tags=uplot_medium_tags, negative_tags=negative_tags)
# print(not_finished.shape)
# not_finished = not_finished[not_finished['sys/state'] != 'Active']
print(not_finished)
not_finished.to_csv('not_finished_relativity.csv')

https://app.neptune.ai/pmtest/llm-random/
table downloaded
shape: (5, 744)
(5, 744)
(5, 744)
                ID  time_running  time_left (hours)  %finished     step  \
0  LLMRANDOM-16028          2.28          63.012096      3.492   1746.0   
1  LLMRANDOM-16020         12.37          32.941355     27.300  13650.0   
2  LLMRANDOM-16005         25.94          18.367041     58.546  29273.0   
3  LLMRANDOM-15082        103.23           0.000000    100.000  50000.0   
4  LLMRANDOM-14973        128.05           0.000000    100.000  50000.0   

             host  args/n_steps  args/model_n_active  args/expansion_rate  \
0         4124gs0         50000                  679                    8   
1  164-152-24-115         50000                  679                    8   
2         login01         50000                  679                    8   
3  164-152-24-115         50000                  679                    8   
4         4124gs0         50000                  679                   